In [2]:
from pyinaturalist.node_api import get_all_observations, get_observations, make_inaturalist_api_get_call
import csv
from pprint import pprint
from tempfile import NamedTemporaryFile
import shutil
import csv

In [3]:
fields = ["name","alt_name","russian_name","wiki_de","latin_name","image_url","inat_id","wiki_en","preferred_name","inat_name"]

In [19]:
birdsfile = 'berlin-birds-extended.csv'
tempfile = NamedTemporaryFile(mode='w', delete=False)
with open(birdsfile, newline='',encoding='utf8') as csvfile,tempfile:
        birdreader = csv.DictReader(csvfile, delimiter=',')
        birdswriter = csv.DictWriter(tempfile, delimiter=';',fieldnames=fields)
        birdswriter.writeheader()
        for row in birdreader:
            latin_name = str(row["latin_name"]).split("/")[0]
            print("Checking {0}".format(latin_name))
            taxa = None
            try:
                taxa = make_inaturalist_api_get_call("taxa",params={"q":row["russian_name"]}).json()["results"][0]
            except Exception:
                taxa = make_inaturalist_api_get_call("taxa",params={"q":row["name"]}).json()["results"][0]
            
            newrow = {"name":row["name"],
                  "alt_name":row["alt_name"],
                   "russian_name":row["russian_name"],
                   "wiki_de":row["wiki_de"],
                   "latin_name":row["latin_name"],
                   "image_url":row["image_url"],
                   "inat_id":taxa["id"] if "id" in taxa else None,
                   "wiki_en":taxa["wikipedia_url"] if "wikipedia_url" in taxa else None,
                   "preferred_name":taxa["preferred_common_name"] if "preferred_common_name" in taxa else None,
                   "inat_name": taxa["name"] if "name" in taxa else None
                  }
            birdswriter.writerow(newrow)
            if taxa:
                print("{0} matching {1}".format(latin_name, taxa["name"]))

Checking Turdus merula
Turdus merula matching Turdus merula
Checking Motacilla alba
Motacilla alba matching Motacilla alba
Checking Falco subbuteo
Falco subbuteo matching Falco subbuteo
Checking Carduelis flammea
Carduelis flammea matching Acanthis flammea
Checking Fulica atra
Fulica atra matching Fulica atra
Checking Parus caeruleus
Parus caeruleus matching Cyanistes caeruleus
Checking Fringilla coelebs
Fringilla coelebs matching Fringilla coelebs
Checking Dendrocops major
Dendrocops major matching Dendrocopos major
Checking Corvus monedula
Corvus monedula matching Corvus monedula
Checking Acrocephalus arundinaceus
Acrocephalus arundinaceus matching Acrocephalus arundinaceus
Checking Garrulus glandarius
Garrulus glandarius matching Cyanocitta cristata
Checking Alcedo atthis
Alcedo atthis matching Alcedo atthis
Checking Pica pica
Pica pica matching Pica pica
Checking Phylloscopus trochilus
Phylloscopus trochilus matching Phylloscopus trochilus
Checking Certhia brachydactyla
Certhia bra

In [20]:
shutil.move(tempfile.name, "berlin-birds-extended-inat.csv")                

'berlin-birds-extended-inat.csv'

In [10]:
!cat berlin-birds-extended-inat.csv

Turdus merula
{'ancestor_ids': [48460, 1, 2, 355675, 3, 7251, 15977, 12705, 12716],
 'ancestry': '48460/1/2/355675/3/7251/15977/12705',
 'atlas_id': None,
 'complete_rank': 'subspecies',
 'complete_species_count': None,
 'current_synonymous_taxon_ids': None,
 'default_photo': {'attribution': '(c) Ashley M Bradford, some rights reserved '
                                  '(CC BY-NC)',
                   'flags': [],
                   'id': 7885442,
                   'license_code': 'cc-by-nc',
                   'medium_url': 'https://static.inaturalist.org/photos/7885442/medium.jpg?1495165836',
                   'original_dimensions': {'height': 800, 'width': 1200},
                   'square_url': 'https://static.inaturalist.org/photos/7885442/square.jpg?1495165836',
                   'url': 'https://static.inaturalist.org/photos/7885442/square.jpg?1495165836'},
 'extinct': False,
 'flag_counts': {'resolved': 1, 'unresolved': 1},
 'iconic_taxon_id': 3,
 'iconic_taxon_name': 'Aves',
 'id': 12716,
 'is_active': True,
 'matched_term': 'Turdus merula',
 'name': 'Turdus merula',
 'observations_count': 21840,
 'parent_id': 12705,
 'preferred_common_name': 'Eurasian Blackbird',
 'rank': 'species',
 'rank_level': 10,
 'taxon_changes_count': 1,
 'taxon_schemes_count': 6,
 'wikipedia_url': 'http://en.wikipedia.org/wiki/Common_blackbird'}

In [22]:
"http://api.inaturalist.org/v1/observations?identified=true&photos=true&identifications=most_agree&quality_grade=research&order=desc&order_by=created_at&taxon_id=41944&place_id=1&per_page=200"

'http://api.inaturalist.org/v1/observations?identified=true&photos=true&identifications=most_agree&quality_grade=research&order=desc&order_by=created_at&taxon_id=41944&place_id=1&per_page=200'

In [4]:
place_id = 7207

In [35]:
birdsfile = 'berlin-birds-extended-inat.csv'
with open(birdsfile, newline='',encoding='utf8') as csvfile:
        birdreader = csv.DictReader(csvfile, delimiter=';')
        for row in birdreader:
            print(row["inat_id"])
            params = {"identified":"true",
                        "photos":"true",
                        "identifications":"most_agree",
                        "quality_grade":"research",
                        "order":"desc",
                        "order_by":"created_at",
                        "popular":"true",
                        "per_page":200}
            params["taxon_id"] = row["inat_id"]
            params["place_id"] = place_id
            print(params)
            obs = get_observations(params=params)
            break
        
        
        

12716
{'identified': 'true', 'photos': 'true', 'identifications': 'most_agree', 'quality_grade': 'research', 'order': 'desc', 'order_by': 'created_at', 'popular': 'true', 'per_page': 200, 'taxon_id': '12716'}


In [36]:
len(obs["results"])

50

In [14]:
species = obs["results"]

In [37]:
phs = []
for result in obs["results"]:
    for photo in result["photos"]:
        phs.append(photo["url"].replace("square","medium"))
len(phs)

92

['https://static.inaturalist.org/photos/38793783/medium.jpeg?1557771254',
 'https://static.inaturalist.org/photos/37924491/medium.jpeg?1556993626',
 'https://static.inaturalist.org/photos/35762459/medium.jpg?1556222071',
 'https://static.inaturalist.org/photos/35243164/medium.jpg?1555733406',
 'https://static.inaturalist.org/photos/34066938/medium.jpeg?1554381250',
 'https://static.inaturalist.org/photos/33913786/medium.jpg?1554157242',
 'https://static.inaturalist.org/photos/33913801/medium.jpg?1554157256',
 'https://static.inaturalist.org/photos/33913812/medium.jpg?1554157269',
 'https://static.inaturalist.org/photos/33913823/medium.jpg?1554157283',
 'https://static.inaturalist.org/photos/33485964/medium.jpg?1553609974',
 'https://static.inaturalist.org/photos/33135342/medium.jpeg?1553087907',
 'https://static.inaturalist.org/photos/32751193/medium.jpg?1552431242',
 'https://static.inaturalist.org/photos/32471377/medium.jpeg?1551966481',
 'https://static.inaturalist.org/photos/324713